## Capstone Project - Battle of Neighbour hood
### Week 3
### Author : Umeshpratap Singh
#### This project is final journey towards IBM -Data Science Certification
It is a capstone proejct to demonstrate my learing in various data science subjects using python learned over course of IBM data science certification


In [72]:
import pandas as pd
import numpy as np


## Part 1 ( week 3) : Data Wrangling from Wikipedia for Toronto 

In [1]:
#  install BeautiulSoup  
# ! pip install requests BeautifulSoup4  # only once


     |████████████████████████████████| 122kB 8.9MB/s eta 0:00:01


In [2]:
# import library 
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [73]:
# function to get URL and point out issue if any 
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    


In [44]:
# creating object
raw_html = simple_get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
len(raw_html)


56520

In [64]:
# using beautifulSoup to convert web site table in to dataframe
soup = BeautifulSoup(raw_html, 'html.parser')

table = soup.find('table', attrs={'class':'wikitable sortable'}) # required table on webiste
table_rows = table.find_all('tr')


res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df_toronto = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighbourhood"])

df_toronto.head()



,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [74]:
# removing not assigined value in Borough
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']
df_toronto.head(5)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [62]:
# to check how many Neighbourhood has not assigned value 
dftest = df_toronto[df_toronto.Neighbourhood == 'Not assigned']
dftest.head()

,Postal Code,Borough,Neighbourhood


#### There are no Neighbourhood which are not assigned 

In [71]:
df_toronto.groupby(['PostalCode']).count()

,Borough,Neighbourhood
PostalCode,,
M1B,1,1
M1C,1,1
M1E,1,1
M1G,1,1
M1H,1,1
...,...,...
M9N,1,1
M9P,1,1
M9R,1,1


#### there are no postal code which is repeated twice and hence no merging of rows requried 

In [68]:
df_toronto.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [70]:
df_toronto.shape # final shape of Data frame 

(103, 3)

### Part II of Week 3 assignment , Finding lat and long of each postal code

In [75]:
# Download file 
!wget -q -O 'toronto_latlon.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [77]:
latlan_df = pd.read_csv('toronto_latlon.csv')
latlan_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [81]:
df_merge_toronto = pd.merge(df_toronto, latlan_df, left_on='PostalCode', right_on='Postal Code')
df_merge_toronto.drop(["Postal Code"], axis = 1, inplace = True) 

df_merge_toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Above is final result required 

In [83]:
df_merge_toronto.shape # cross checking if all rows are there 

(103, 5)